In [ ]:
!pip install pandas requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlparse

base_url = 'https://opportunitiesforyoungkenyans.co.ke'
pagination_date_pattern = base_url + '/2024/06/13'
pagination_pattern = pagination_date_pattern + '/page/3'

headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [ ]:
def get_post_body(url, headers):
  response = requests.get(url, headers=headers)

  if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    section = soup.find('section', class_='main-content')

    if section:
      post_article = section.find('article', class_='post-content')

      if post_article:
        post_container = post_article.find('div', class_='clearfix').text
        return post_container

In [ ]:
def extract_date_from_url(url):
  parsed_url =  urlparse(url)
  
  path_parts = parsed_url.path.split('/')
  
  year = path_parts[1]
  month = path_parts[2]
  day = path_parts[3]
  
  date = f'{year}-{month}-{day}'
  return date

In [ ]:
def get_job_postings(url, headers):
  response = requests.get(url, headers=headers)

  if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    section = soup.find('section', class_='main-content')

    if section:
      post_container = section.find('div', class_='col-lg-8')

      job_posts = []
      
      for post in post_container.find_all('div', class_='post-classic'):
        title_heading_element = post.find('h5')

        title = title_heading_element.find('a').text
        link = title_heading_element.find('a').get("href")

        content = get_post_body(link, headers)
        date = extract_date_from_url(link)

        job_posts.append({
          'title': title,
          'link': link,
          'content': content
        })

      return job_posts
  else:
    return []

all_jobs = get_job_postings(pagination_date_pattern, headers)
df = pd.DataFrame(all_jobs)
df.to_csv('job_postings.csv', index=False)